# Image classification with small data

1. Download and setup dataset (Kaggle Cats & Dogs) https://www.kaggle.com/c/dogs-vs-cats/data
2. Train a small convnet on our small data (acc ~ 0.75)
3. Train a MLP using bottleneck features of pretrained model (acc ~ 0.90)
4. Finetuning top layers of pretrained model on our small data (acc ~ 0.94)


In [47]:
# imports
!KERAS_BACKEND=tensorflow python -c "from keras import backend"
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import applications
import numpy as np
from keras import Model
from keras import optimizers
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [48]:
# define plot & parse functions
def parse_log_file(pfile):
    f= open(pfile).readlines()
    lines = [i.strip() for i in f]
    log= {k:[] for k in ['acc','val_acc','loss','val_loss']}
    for l in lines:
        if 'loss' in l:
            for k in log.keys():
                log[k].append(float(l.split(k+': ')[1].split(' -')[0]))
    return log 

def plot_loss_acc(pfile):
    history = parse_log_file(pfile)
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.title('model acc')
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [49]:
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)

## Train a small convnet on our small data

In [50]:
# define a sequential model (small conv net): 
# 3 conv blocks (Conv2D, Activation('relu'), MaxPooling2D) + 2 dense layers
# Conv_1: filters 32, kernel size(3,3)
# Conv_2: filters 32, kernel size(3,3)
# Conv_3: filters 64, kernel size(3,3)
# Flatten
# Dense_1: 64
# Activation('relu')
# Dropout(0.5)
# Dense_2:  ? 
# Activation('sigmoid')


model = Sequential(layers=[
    Conv2D(32, (3,3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(),
    Conv2D(32, (3,3)),
    Activation('relu'),
    MaxPooling2D(),
    Conv2D(64, (3,3)),
    Activation('relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5),
    Dense(1),
    Activation('sigmoid')]
)
    
    
# print model summary
model.summary()
# # add your implementation

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_18 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_19 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [51]:
# compile model 
# use binary crossentropy loss 
# and rmsprop optimizer

# # add your implementation

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [52]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [53]:
# define a keras ImageDataGenerator for training data with appropriate augmentation 
# use rescale=1. / 255 to normalise pixles values

img_generator_training = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1./255,)

# define a keras ImageDataGenerator for test data  (no augmentation only rescaling)

img_generator_test = ImageDataGenerator(
    rescale=1./255)


In [54]:
# define training and validation iterators 
# use ImageDataGenerator.flow_from_directory to training and validation dirs accordingly 

# # add your implementation

training_iterator = img_generator_training.flow_from_directory(train_data_dir, target_size=(150,150), 
                                                               class_mode='binary')
test_iterator = img_generator_test.flow_from_directory(validation_data_dir, target_size=(150,150), 
                                                       class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [55]:
# train and validate the model using fit_generator

# # add your implementation

model.fit_generator(training_iterator, steps_per_epoch=nb_train_samples // batch_size, 
                    validation_data=test_iterator, epochs=50, 
                    validation_steps= nb_validation_samples // batch_size)

model.save_weights()

Epoch 1/50
  1/125 [..............................] - ETA: 3:14 - loss: 0.6945

KeyboardInterrupt: 

## Train a MLP using bottleneck features

In [56]:
# define VGG16 network using keras applications.VGG16  
# set weights = 'imagenet'
# set include_top=False : don't include the fully-connected layer at the top of the network 
# set input_shape: image shape

from keras.applications import VGG16
network = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
# # add your implementation

# define a keras ImageDataGenerator for data (no augmentation only rescaling)
# define training and validation iterators same as before (set shuffle=False)

img_generator_small = ImageDataGenerator(
    rescale=1./255,)

training_iterator = img_generator_small.flow_from_directory(train_data_dir, target_size=(150,150), 
                                                            shuffle=False, batch_size=batch_size, class_mode=None)
test_iterator = img_generator_small.flow_from_directory(validation_data_dir, target_size=(150,150), shuffle=False)

# # add your implementation

# extract image features for training and validation separately using predict_generator

features_training = network.predict_generator(training_iterator, steps=nb_train_samples // batch_size)
features_test = network.predict_generator(test_iterator, steps = nb_validation_samples // batch_size)

# # add your implementation

# save training and validation features

# # add your implementation

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


KeyboardInterrupt: 

In [57]:
features_training = np.load('bottleneck_features_train.npy')
features_test = np.load('bottleneck_features_validation.npy')

In [60]:
features_training.shape

(2000, 4, 4, 512)

In [87]:
# create training and validation labels array (features are in order because we used shuffle=false) 

# # add your implementation

labels_training = np.vstack((np.zeros((1000, 1)), np.ones((1000, 1))))
labels_validation = np.vstack((np.zeros((400, 1)), np.ones((400, 1))))

In [86]:
labels_training.shape

(2000, 1)

In [88]:
# define model of two dense layers 256 and ? 
# model should start with Flatten layer (to flatten extracted image features to a vector)
# don’t forget activations and dropout(0.5)
# compile with binary_crossentropy loss and rmsprop optimizer

# # add your implementation

#train_preds = np.load('bottleneck_features_train.npy')
#test_preds = np.load('bottleneck_features_validation.npy')

MLP = Sequential(layers=[
    Flatten(input_shape=(4, 4, 512)),
    Dense(256),
    Activation('relu'),
    Dropout(0.5),
    Dense(1),
    Activation('sigmoid')]
)

MLP.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [89]:
MLP.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_14 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 256)               2097408   
_________________________________________________________________
activation_33 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 257       
_________________________________________________________________
activation_34 (Activation)   (None, 1)                 0         
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
_________________________________________________________________


In [90]:
# Train MLP using fit function

# # add your implementation


W = MLP.fit(x=features_training, y=labels_training, steps_per_epoch=nb_train_samples // batch_size, 
            validation_data=(features_test, labels_validation), validation_steps=nb_validation_samples//batch_size,
            epochs=10)

MLP.fit()
# save weights to use them in fine-tuning later on

# # add your implementation

Train on 2000 samples, validate on 800 samples
Epoch 1/10
125/125 [==============================] - 31s 248ms/step - loss: 0.4551 - acc: 0.8493 - val_loss: 0.4086 - val_acc: 0.8213 ETA: 1s - loss: 0.4694 - acc: 
Epoch 2/10
125/125 [==============================] - 34s 269ms/step - loss: 0.1083 - acc: 0.9579 - val_loss: 0.2874 - val_acc: 0.9112
Epoch 3/10
125/125 [==============================] - 37s 297ms/step - loss: 0.0757 - acc: 0.9797 - val_loss: 0.5459 - val_acc: 0.7925
Epoch 4/10
125/125 [==============================] - 32s 260ms/step - loss: 0.0593 - acc: 0.9889 - val_loss: 0.2722 - val_acc: 0.9125
Epoch 5/10
125/125 [==============================] - 35s 277ms/step - loss: 0.0400 - acc: 0.9956 - val_loss: 0.4674 - val_acc: 0.9125
Epoch 6/10
125/125 [==============================] - 42s 333ms/step - loss: 0.0548 - acc: 0.9906 - val_loss: 0.4676 - val_acc: 0.9075
Epoch 7/10
125/125 [==============================] - 44s 353ms/step - loss: 0.0308 - acc: 0.9950 - val_loss: 0.

ValueError: If fitting from data tensors, you should specify the `steps_per_epoch` argument.

## Finetuning top layers of pretrained model on our small data

In [ ]:
# define new model:  VGG16 as base and MLP as top 
# input = base_model.input, output=top_model(base_model.output)

# # add your implementation

# freeze the first 15 layers (up to the last conv block)
# set trainable=false (weights will not be updated)

# # add your implementation

# compile with binary_crossentropy loss and SGD with low learing rate optimizer

# # add your implementation

In [ ]:
# fine-tune the model using fit_generator and train and validation iterators